# Bermudan Swaption Pricing

Value a European Swaption using the Hull-White, Black-Derman-Toy and Black-Karasinski models. I analyse relationships to test the model pricing.

In [140]:
import numpy as np
import matplotlib.pyplot as plt

In [141]:
from financepy.finutils import *
from financepy.products.libor import *
from financepy.market.curves import *
from financepy.models.FinModelRatesHW import FinModelRatesHW
from financepy.models.FinModelRatesBK import FinModelRatesBK
from financepy.models.FinModelRatesBDT import FinModelRatesBDT

## Discount Curve

In [142]:
valuationDate = FinDate(1, 1, 2011)
settlementDate = valuationDate

Set up flat discount curve

In [143]:
rate = 0.0625
liborCurve = FinDiscountCurveFlat(valuationDate, rate, FinFrequencyTypes.SEMI_ANNUAL)

# The Underlying Swap

We begin with the underlying swap.

In [144]:
exerciseDate = settlementDate.addTenor("1Y")
swapMaturityDate = settlementDate.addTenor("4Y")
swapFixedCoupon = 0.060
swapFixedFrequencyType = FinFrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = FinDayCountTypes.ACT_365F
swapType = FinLiborSwapTypes.PAYER 

We can value the forward starting swap.

In [145]:
swap = FinLiborSwap(exerciseDate,
                    swapMaturityDate,
                    swapType,
                    swapFixedCoupon,
                    swapFixedFrequencyType,
                    swapFixedDayCountType)

In [146]:
swapValue = swap.value(valuationDate, liborCurve, liborCurve)
print("Swap Value: %9.5f"% swapValue)

Swap Value: 6193.31147


# Bermudan Swaption Framework

I now examine FinBermudanSwaption and use the tree models to value it. But first let me price it on the tree models as a European option.

In [147]:
blackVol = 0.0000020

## Start By Only Allowing European Exercise

In [148]:
europeanSwaptionPay = FinLiborBermudanSwaption(settlementDate,
                                               exerciseDate,
                                               swapMaturityDate,
                                               FinLiborSwapTypes.PAYER,
                                               FinOptionExerciseTypes.EUROPEAN,
                                               swapFixedCoupon,
                                               swapFixedFrequencyType,
                                               swapFixedDayCountType)

In [149]:
europeanSwaptionRec = FinLiborBermudanSwaption(settlementDate,
                                               exerciseDate,
                                               swapMaturityDate,
                                               FinLiborSwapTypes.RECEIVER,
                                               FinOptionExerciseTypes.EUROPEAN,
                                               swapFixedCoupon,
                                               swapFixedFrequencyType,
                                               swapFixedDayCountType)

### Black Karasinski Model

In [150]:
sigma = blackVol
a = 0.01
numTimeSteps = 101
model = FinModelRatesBK(sigma, a, numTimeSteps)

In [151]:
valuePay = europeanSwaptionPay.value(valuationDate, liborCurve, model)
valueRec = europeanSwaptionRec.value(valuationDate, liborCurve, model)
valuePayRec = valuePay - valueRec
print("Payer Value: %9.5f"% valuePay)
print("Recvr Value: %9.5f"% valueRec)
print("PayV - RecV: %9.5f"% valuePayRec)

Payer Value: 6193.31120
Recvr Value:   0.00000
PayV - RecV: 6193.31120


### Black Derman Toy Model

In [152]:
sigma = blackVol
numTimeSteps = 101
model = FinModelRatesBDT(sigma, numTimeSteps)

In [153]:
valuePay = europeanSwaptionPay.value(valuationDate, liborCurve, model)
valueRec = europeanSwaptionRec.value(valuationDate, liborCurve, model)
valuePayRec = valuePay - valueRec
print("Payer Value: %9.5f"% valuePay)
print("Recvr Value: %9.5f"% valueRec)
print("PayV - RecV: %9.5f"% valuePayRec)

Payer Value: 6192.84909
Recvr Value:   0.00000
PayV - RecV: 6192.84909


### Hull White Model

In [154]:
sigma = blackVol * rate
a = 0.01
numTimeSteps = 101
model = FinModelRatesHW(sigma, a, numTimeSteps)

In [155]:
valuePay = europeanSwaptionPay.value(valuationDate, liborCurve, model)
valueRec = europeanSwaptionRec.value(valuationDate, liborCurve, model)
valuePayRec = valuePay - valueRec
print("Payer Value: %9.5f"% valuePay)
print("Recvr Value: %9.5f"% valueRec)
print("PayV - RecV: %9.5f"% valuePayRec)

Payer Value: 6193.31147
Recvr Value:   0.00000
PayV - RecV: 6193.31147


# Now allow Multiple Exercise Dates - Bermudan

Now allow exercise on coupon dates after expiry.

In [156]:
bermudanSwaptionPay = FinLiborBermudanSwaption(settlementDate,
                                               exerciseDate,
                                               swapMaturityDate,
                                               FinLiborSwapTypes.PAYER,
                                               FinOptionExerciseTypes.BERMUDAN,
                                               swapFixedCoupon,
                                               swapFixedFrequencyType,
                                               swapFixedDayCountType)

In [157]:
bermudanSwaptionRec = FinLiborBermudanSwaption(settlementDate,
                                               exerciseDate,
                                               swapMaturityDate,
                                               FinLiborSwapTypes.RECEIVER,
                                               FinOptionExerciseTypes.BERMUDAN,
                                               swapFixedCoupon,
                                               swapFixedFrequencyType,
                                               swapFixedDayCountType)

### Black Karasinski Model

In [158]:
sigma = 0.20
a = 0.01
numTimeSteps = 201
model = FinModelRatesBK(sigma, a, numTimeSteps)

In [159]:
valuePay = bermudanSwaptionPay.value(valuationDate, liborCurve, model)
valueRec = bermudanSwaptionRec.value(valuationDate, liborCurve, model)
valuePayRec = valuePay - valueRec
print("Payer Value: %9.5f"% valuePay)
print("Recvr Value: %9.5f"% valueRec)
print("PayV - RecV: %9.5f"% valuePayRec)

Payer Value: 19079.53200
Recvr Value: 13065.67271
PayV - RecV: 6013.85930


### Black-Derman-Toy Model

In [160]:
sigma = 0.20
numTimeSteps = 201
model = FinModelRatesBDT(sigma, numTimeSteps)

In [161]:
valuePay = bermudanSwaptionPay.value(settlementDate, liborCurve, model)
valueRec = bermudanSwaptionRec.value(settlementDate, liborCurve, model)
valuePayRec = valuePay - valueRec
print("Payer Value: %9.5f"% valuePay)
print("Recvr Value: %9.5f"% valueRec)
print("PayV - RecV: %9.5f"% valuePayRec)

Payer Value: 19337.67907
Recvr Value: 13356.33735
PayV - RecV: 5981.34172


### Hull-White Model

In [162]:
sigma = 0.01
a = 0.01
numTimeSteps = 201
model = FinModelRatesHW(sigma, a)

In [163]:
valuePay = bermudanSwaptionPay.value(settlementDate, liborCurve, model)
valueRec = bermudanSwaptionRec.value(settlementDate, liborCurve, model)
valuePayRec = valuePay - valueRec
print("Payer Value: %9.5f"% valuePay)
print("Recvr Value: %9.5f"% valueRec)
print("PayV - RecV: %9.5f"% valuePayRec)

Payer Value: 16468.31966
Recvr Value: 10517.20803
PayV - RecV: 5951.11163


The swaption is Bermudan so it can only be exercised on coupon payment dates

Copyright (c) 2020 Dominic O'Kane